In [1]:
import requests
import simplejson as json
from tqdm import tqdm
from openpyxl import load_workbook

import os
from glob import glob
import csv
import sys

In [2]:
# If reports don't land in a specific spot on the web server, or a related project folder,
# assume reports should land in a subfolder of the current directory

reportdir = "/var/www/html/misc/20200417-covid-county-analysis/"   # Blank or ending in a slash
if not os.path.exists(reportdir):
    reportdir = "../covid-counties-analysis/"
    if not os.path.exists(reportdir):
        reportdir = ""

xlsxdir = "xlsx/"
csvdir = "csv/"
summarydir = reportdir + "white-house-reports/"
ignoredir = reportdir + "ignore/"
highlightsfile = ignoredir + "white-house-latest.json"

for targetdir in [xlsxdir, csvdir, summarydir, ignoredir]:
    os.makedirs(targetdir, exist_ok=True)

In [3]:
baseurl = "https://healthdata.gov"
starturl = "https://healthdata.gov/Health/COVID-19-Community-Profile-Report/gqxm-d9w9"
r = requests.get(starturl)
html = r.text

In [4]:
# Find the right JSON object
for row in html.splitlines():
    if '{"view":' in row:
        break

In [5]:
rawdata = json.loads(row)

In [6]:
# Grab just what we want
entries = rawdata['view']['attachments']

In [7]:
newfiles = 0
for entry in tqdm(entries):
    basefilename = entry['name']
    if ".xlsx" in basefilename:   # Get just the Excel files
        if " " in basefilename:    # sample: 'Community Profile Report 20220207.xlsx'
            print(f"Old basefilename: {basefilename}")
            keydate = basefilename.split(" ")[-1].split(".xlsx")[0]
            basefilename = f"Community_Profile_Report_{keydate}_Public.xlsx"
            print(f"New basefilename: {basefilename}")
        if "Public" not in basefilename:
            print(f"Old basefilename: {basefilename}")
            basefilename = basefilename.replace(".xlsx", "_Public.xlsx")                
            print(f"New basefilename: {basefilename}")
        targetfilename = xlsxdir + basefilename
        if "_P_" not in targetfilename:
            if not os.path.exists(targetfilename):    # if we haven't already downloaded this
                filedate = basefilename.split("_")[-2]
                if not os.path.exists(csvdir + filedate + ".csv"):    # If we don't have the CSV either
                    targeturl = baseurl + entry['href']
                    r = requests.get(targeturl)
                    if r.status_code != 200:
                        print(f"Error downloading {basefilename} from {targeturl}")
                    else:
                        with open(targetfilename, "wb") as outfile:
                            outfile.write(r.content)
                        newfiles += 1
print(f"{newfiles} new file(s) found")

100%|██████████████████████████████████████████████████████████████████████████████| 736/736 [00:00<00:00, 9184.02it/s]

Old basefilename: Community_Profile_Report_20220207.xlsx
New basefilename: Community_Profile_Report_20220207_Public.xlsx
0 new file(s) found


In [8]:
if newfiles == 0:
    print("No new files found. Stopping.")
    sys.exit()

No new files found. Stopping.


SystemExit: 

C:\Python37\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
excelfiles = glob(xlsxdir + "*.xlsx")
for excelfilename in tqdm(excelfiles):
    filedate = excelfilename.split("_")[-2]
    csvfilename = csvdir + filedate + ".csv"
    if not os.path.exists(csvfilename):
        workbook = load_workbook(filename=excelfilename)
        countytab = workbook["Counties"]
        with open(csvfilename, "w", newline="", encoding="utf-8") as outfile:
            writer = csv.writer(outfile)
            for entry in list(countytab.iter_rows(values_only=True))[1:]:   # Skip first row
                writer.writerow(list(entry))


In [ ]:
masterdict = {}
csvfiles = list(sorted(glob(csvdir + "*.csv")))
for filenumber, csvfilename in enumerate(tqdm(csvfiles)):
    basefilename = csvfilename.replace("\\", "/").replace(csvdir, "")
    filedate = basefilename.replace(".csv", "")
    with open(csvfilename, "r", encoding="utf-8") as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            fips = row['FIPS code'].zfill(5)
            state = row['State Abbreviation']
            county = row['County']
            cases = row['Cumulative cases']
            deaths = row['Cumulative deaths']
            if cases == "":
                cases = 0
            else:
                cases = int(cases)
            if deaths == "":
                deaths = 0
            else:
                deaths = int(deaths)
            if state not in masterdict:
                masterdict[state] = {}
            if fips not in masterdict[state]:
                masterdict[state][fips] = {}
            line = {
                "fips": fips,
                "state": state,
                "county": county,
                "filedate": filedate,
                "cases": cases,
                "deaths": deaths
            }
            masterdict[state][fips][filedate] = line   

In [ ]:
headers = list(masterdict[state][fips][filedate].keys())
for state in tqdm(masterdict):
    with open(summarydir + state + ".csv", "w", newline="", encoding="utf-8") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(headers)
        for fips in sorted(masterdict[state]):
            for filedate in masterdict[state][fips]:
                writer.writerow(list(masterdict[state][fips][filedate].values()))

In [ ]:
highlightsdict = {}
for state in masterdict:
    for fips in masterdict[state]:
        localdates = list(sorted(masterdict[state][fips].keys()))
        if fips in highlightsdict:
            print(f"FIPS {fips} was repeated somehow.")
        highlightsdict[fips] = masterdict[state][fips][localdates[-1]]

In [ ]:
temp = {}
for fips in list(sorted(highlightsdict.keys())):
    temp[fips] = highlightsdict[fips]
highlightsdict = temp
temp = None

In [ ]:
with open(highlightsfile, "w") as outfile:
    outfile.write(json.dumps(highlightsdict, indent=" " * 4))